In [16]:
import sqlite3

conn = sqlite3.connect("flights.db")
cur = conn.cursor()

In [17]:
cur.execute("""CREATE TABLE Passenger(
  ID_psg INTEGER,
  name VARCHAR(100))""")

In [18]:
cur.execute("""INSERT INTO Passenger
VALUES (1, 'Bruce Willis'),
       (2, 'George Clooney'),
       (3, 'Kevin Costner'),
       (4, 'Donald Sutherland'),
       (5, 'Jennifer Lopez'),
       (6, 'Ray Liotta'),
       (7, 'Samuel L. Jackson'),
       (8, 'Nikole Kidman'),
       (9, 'Alan Rickman'),
       (10, 'Kurt Russell'),
       (11, 'Harrison Ford'),
       (12, 'Russell Crowe'),
       (13, 'Steve Martin'),
       (14, 'Michael Caine'),
       (15, 'Angelina Jolie'),
       (16, 'Mel Gibson'),
       (17, 'Michael Douglas'),
       (18, 'John Travolta'),
       (19, 'Sylvester Stallone'),
       (20, 'Tommy Lee Jones'),
       (21, 'Catherine Zeta-Jones'),
       (22, 'Antonio Banderas'),
       (23, 'Kim Basinger'),
       (24, 'Sam Neill'),
       (25, 'Gary Oldman'),
       (26, 'Clint Eastwood'),
       (27, 'Brad Pitt'),
       (28, 'Johnny Depp'),
       (29, 'Pierce Brosnan'),
       (30, 'Sean Connery'),
       (31, 'Bruce Willis'),
       (37, 'Mullah Omar')""")

In [19]:
conn.commit()

In [20]:

cur.execute("""CREATE TABLE Pass_in_trip (
  trip_no INTEGER,
  date DATETIME,
  ID_psg INTEGER,
  place VARCHAR(100)
)""")

cur.execute("""INSERT INTO Pass_in_trip
VALUES (1100, '2003-04-29 00:00:00.000', 1, '1a'),
       (1123, '2003-04-05 00:00:00.000', 3, '2a'),
       (1123, '2003-04-08 00:00:00.000', 1, '4c'),
       (1123, '2003-04-08 00:00:00.000', 6, '4b'),
       (1124, '2003-04-02 00:00:00.000', 2, '2d'),
       (1145, '2003-04-05 00:00:00.000', 3, '2c'),
       (1181, '2003-04-01 00:00:00.000', 1, '1a'),
       (1181, '2003-04-01 00:00:00.000', 6, '1b'),
       (1181, '2003-04-01 00:00:00.000', 8, '3c'),
       (1181, '2003-04-13 00:00:00.000', 5, '1b'),
       (1182, '2003-04-13 00:00:00.000', 5, '4b'),
       (1187, '2003-04-14 00:00:00.000', 8, '3a'),
       (1188, '2003-04-01 00:00:00.000', 8, '3a'),
       (1182, '2003-04-13 00:00:00.000', 9, '6d'),
       (1145, '2003-04-25 00:00:00.000', 5, '1d'),
       (8882, '2005-11-06 00:00:00.000', 37, '1a'),
       (7771, '2005-11-07 00:00:00.000', 37, '1c'),
       (7772, '2005-11-07 00:00:00.000', 37, '1a'),
       (8881, '2005-11-08 00:00:00.000', 37, '1d'),
       (7778, '2005-11-05 00:00:00.000', 10, '2a'),
       (7772, '2005-11-29 00:00:00.000', 10, '3a'),
       (7771, '2005-11-04 00:00:00.000', 11, '4a'),
       (7771, '2005-11-07 00:00:00.000', 11, '1b'),
       (7771, '2005-11-09 00:00:00.000', 11, '5a'),
       (7772, '2005-11-07 00:00:00.000', 12, '1d'),
       (7773, '2005-11-07 00:00:00.000', 13, '2d'),
       (7772, '2005-11-29 00:00:00.000', 13, '1b'),
       (8882, '2005-11-13 00:00:00.000', 14, '3d'),
       (7771, '2005-11-14 00:00:00.000', 14, '4d'),
       (7771, '2005-11-16 00:00:00.000', 14, '5d'),
       (7772, '2005-11-29 00:00:00.000', 14, '1c'),
       (1187, '2003-04-14 00:00:00.000', 10, '3d')""")

In [21]:
conn.commit()

In [22]:
cur.execute("""
WITH flight_counts AS (
    SELECT p.ID_psg, p.name, COUNT(pt.trip_no) AS flight_count
    FROM Passenger p
    LEFT JOIN Pass_in_trip pt ON p.ID_psg = pt.ID_psg
    GROUP BY p.ID_psg, p.name
),
max_flights AS (
    SELECT MAX(flight_count) AS max_cnt FROM flight_counts
),
ordered AS (
    SELECT
        ID_psg,
        name,
        flight_count,
        LAG(name) OVER (ORDER BY ID_psg)  AS prev_name,
        LEAD(name) OVER (ORDER BY ID_psg) AS next_name,
        FIRST_VALUE(name) OVER (ORDER BY ID_psg
                                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS first_name,
        LAST_VALUE(name)  OVER (ORDER BY ID_psg
                                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_name
    FROM flight_counts
)
SELECT
    name,
    COALESCE(prev_name, last_name) AS prev_passenger,
    COALESCE(next_name, first_name) AS next_passenger
FROM ordered o
JOIN max_flights m ON o.flight_count = m.max_cnt
""").fetchall()

[('Michael Caine', 'Steve Martin', 'Angelina Jolie'),
 ('Mullah Omar', 'Bruce Willis', 'Bruce Willis')]